# EDA 

This notebook cleans the Chocolate Sales dataset and creates 1-2 static visuals to support our dashboard user story (sales trends over time by country).

## Why we do these checks

Before making plots, we do quick data checks that keep the dashboard stable: we confirm data types, check missing values and duplicates, and make sure key fields like Date and Amount are usable. We also create simple Year/Month fields because most dashboard views (monthly trends and filtering) depend on them.